In [1]:
#tratamento de dados
import pandas as pd
import numpy as np

#visualização de dados
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly.offline as py
import seaborn as sns

#dash
import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

py.init_notebook_mode(connected=True)


import warnings
warnings.filterwarnings('ignore')

# Links Uteis

- https://plotly.com/python/plotly-fundamentals/

In [20]:
#cria um banco de dados aleatorio
waterfall = pd.DataFrame(np.random.randint(-50,75,size=(1000, 6)), 
                  columns=["Sales", "Consulting", "Net revenue", "Purchases", "Other expenses", "Profit before tax"])

In [21]:
waterfall.head()

,Sales,Consulting,Net revenue,Purchases,Other expenses,Profit before tax
0,2,20,-24,-40,-50,-41
1,51,74,41,63,55,8
2,-14,53,53,69,11,0
3,6,37,-40,-27,-38,12
4,45,0,8,21,-22,0


# Watterfall

In [24]:
def waterfall(df, titulo):
    '''
    INPUT:
    df - banco de dados apenas com colunas que deseja analisar e linhas com valores inteiros
    titulo - (str) titulo do gráfico
    
    OUTPUT:
    html_file - salva o gráfico em um arquivo html no local onde está o código
    Gráfico de cascata
    
    '''
          
    #transforma as colunas em linhas para conseguir criar o waterfall
    df = df.melt().groupby('variable', as_index=False)['value'].sum()
    
    #acrescenta uma linha de total no fim
    df.loc[len(df.index)] = ['Total', df['value'].sum()]
    
    #define as variaveis
    x = df['variable'].tolist()
    y = df['value'].tolist()
    
    #cria o gráfico
    fig = go.Figure(go.Waterfall(
        name = "1", orientation = "v",
        #variaveis
        x = x,
        y = y,
        #anotações e posicionamento
        text = y,
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
    ))

    fig.update_layout(
        title = titulo, #titulo
        showlegend = False,
        
        #define as cores
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
    )
        
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside')
    
    #y axis and x axis   
    fig.update_yaxes(visible=False)

    fig.show()
    
    html_file = fig.write_html("./watterfall.html")
    
    return html_file

In [23]:
waterfall(waterfall, 'waterfall')

AttributeError: 'function' object has no attribute 'melt'

In [31]:
estados = ['RS', 'PR', 'SC',
           'SP', 'RJ', 'MG', 'ES',
           'PA', 'AM', 'RO', 'TO', 'AC', 'AP', 'RR',
           'BA', 'CE', 'PE', 'MA', 'RN', 'PB', 'SE', 'AL', 'PI',
           'MS', 'MT', 'GO', 'DF'
          ]

regiao = ['Sul', 'Sul', 'Sul',
          'Sudeste', 'Sudeste', 'Sudeste', 'Sudeste',
          'Norte', 'Norte', 'Norte', 'Norte', 'Norte', 'Norte', 'Norte',
          'Nordeste', 'Nordeste', 'Nordeste', 'Nordeste', 'Nordeste', 'Nordeste', 'Nordeste', 'Nordeste', 'Nordeste',
          'Centro-Oeste', 'Centro-Oeste', 'Centro-Oeste', 'Centro-Oeste'
         ]

# Barras Empilhadas

A estrutura dos dados precisa ser trabalhada da seguinte maneira:

In [32]:
#Cria um banco de dados com duas colunas genéricas
df = pd.DataFrame(columns=['Estado', 'Regiao'])
df['Estado'] = estados #define os valores
df['Regiao'] = regiao #define os valores
df['B3'] = np.random.randint(20000, 40000, size=len(df)) #cria valores numéricos aleatórios
df['Empresa'] = np.random.randint(-2000, 15000, size=len(df)) #cria valores numéricos aleatórios

df['B3_perc'] = round(df['B3']/sum(df['B3'])*100, 2) #cria uma coluna de proporção
df['Empresa_perc'] = round(df['Empresa']/sum(df['Empresa'])*100, 2) #cria uma coluna de proporção

#organiza os dados
df = pd.melt(df, id_vars=['Estado', 'Regiao'], value_vars=['B3_perc', 'Empresa_perc'], ignore_index=False)
df = df.groupby(['Regiao', 'variable'], as_index=False)['value'].sum()

In [33]:
df.head()

,Regiao,variable,value
0,Centro-Oeste,B3_perc,15.78
1,Centro-Oeste,Empresa_perc,13.34
2,Nordeste,B3_perc,30.25
3,Nordeste,Empresa_perc,36.05
4,Norte,B3_perc,28.92


In [34]:
def stacked_bars(df, titulo):
    '''
    INPUT:
    df - banco de dados apenas com colunas que deseja analisar e linhas com valores inteiros
    titulo - (str) titulo do gráfico
    
    OUTPUT:
    html_file - salva o gráfico em um arquivo html no local onde está o código
    Gráfico de cascata    
    
    
    '''

    fig = px.bar(df_melt, x='variable', y='value',
                 text='value',
                 color='Regiao',
                 color_discrete_map={
                     'Centro-Oeste': 'lightgrey',
                     'Nordeste': '#1957E6',
                     'Norte': '#1957E6',
                     'Sudeste': 'lightgrey',
                     'Sul': 'lightgrey'
                 })

    fig.update_layout(
        title=titulo,
        showlegend=True,
        legend_traceorder='reversed',    
        #define as cores
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        uniformtext_minsize=12,
        uniformtext_mode='hide'
    )

    fig.update_traces(texttemplate='%{text:.02s}%',
                      textposition='auto'
                     )

    #y axis and x axis   
    fig.update_yaxes(visible=False)
    fig.update_xaxes(visible=True)

    fig.show()
    
    html_file = fig.write_html("./stacked_bar.html")
    
    return html_file

In [35]:
stacked_bars(df, 'Comparação entre duas bases')

NameError: name 'df_melt' is not defined